# **Library**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl
import os 
import imageio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import ntpath
import csv
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from utils import generator, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from random import shuffle

# **Train model**

In [3]:
BATCH_SIZE = 32
EPOCHS = 20
samples = []
distribution=[]

In [4]:
# open file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
        distribution.append(float(line[3]))

In [ ]:
#plot data
n, bins, patches = plt.hist(np.array(distribution), bins=20)
plt.show()

In [9]:
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [10]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=BATCH_SIZE)
validation_generator = generator(validation_samples, batch_size=BATCH_SIZE, is_training=False)

### ***MODEL***

In [11]:
def basic_model ():

    model = tf.keras.Sequential([
        
        tfl.Conv2D (24, (5,5), strides = (2,2), activation = 'elu' ),
        tfl.BatchNormalization (),
        tfl.Conv2D (36, (5,5), strides = (2,2), activation = 'elu'),
        tfl.BatchNormalization(),
        tfl.Conv2D (48, (5,5), strides = (2,2), activation = 'elu'),
        tfl.BatchNormalization (),
        tfl.Conv2D(64, (3,3), activation = 'elu'),
        tfl.BatchNormalization(),
        tfl.Conv2D(64, (3,3), activation = 'elu'),
        tfl.BatchNormalization(),

        tfl.Flatten(),
        tfl.Dense (100, activation = 'elu'),
        tfl.Dropout(0.5),
        tfl.BatchNormalization(),
        tfl.Dense (50, activation = 'elu'),
        tfl.Dropout (0.5),
        tfl.BatchNormalization(),
        tfl.Dense (10, activation = 'elu'),
        tfl.Dense(1)
    ])
    return model

In [ ]:
model = basic_model()
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

In [13]:
checkpoint = tf.keras.callbacks.ModelCheckpoint ('checkpoints/model-{epoch:02d}-{val_loss:.2f}.h5',
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=False,
                                 mode='auto')

In [ ]:
model.fit(train_generator,epochs = EPOCHS, batch_size = BATCH_SIZE, callbacks = [checkpoint])

In [ ]:
# model.save('model.h5')